In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from ast import literal_eval
from tqdm import tqdm
from sklearn.metrics import classification_report
tqdm.pandas()
import random
random.seed(0)
from belt_nlp.bert_with_pooling import BertClassifierWithPooling

In [2]:
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'
reports_path = '../reports/'
file_format_users_filtered = processed_data_path + 'r3_{target}_{split}_users_scored_Timeline.csv' 
file_format_tmt_filtered = processed_data_path + '{split}_r3_{target}_top_mentioned_timelines_scored_Texts.csv'

In [3]:
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
    ]

In [4]:
dict_experiments = {
    'filtered_Texts5': {
        "text_col": 'Texts',
        "n_comments": 5,
        "file_format": file_format_tmt_filtered
    },
    'filteredTimeline5': {
        "text_col": 'Timeline',
        "n_comments": 5,
        "file_format": file_format_users_filtered
    },
    'filtered_Texts10': {
        "text_col": 'Texts',
        "n_comments": 10,
        "file_format": file_format_tmt_filtered
    },
    'filteredTimeline10': {
        "text_col": 'Timeline',
        "n_comments": 10,
        "file_format": file_format_users_filtered
    },
    'filtered_Texts15': {
        "text_col": 'Texts',
        "n_comments": 15,
        "file_format": file_format_tmt_filtered
    },
    'filteredTimeline15': {
        "text_col": 'Timeline',
        "n_comments": 15,
        "file_format": file_format_users_filtered
    },
}

# Example - Model BERT with pooling

In this notebook we will show how to use basic methods `fit` and `predict` for the BERT model with pooling.

In [5]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [6]:
bert_model_name = 'pablocosta/bertabaporu-base-uncased'
epochs = 3
batch_size = 3

MODEL_PARAMS = {
    "batch_size": batch_size,
    "learning_rate": 5e-5,
    "epochs": epochs,
    "chunk_size": 510,
    "stride": 510,
    "minimal_chunk_length": 510,
    "pooling_strategy": "mean",
    "pretrained_model_name_or_path": bert_model_name
}

In [7]:
bert_model_name.replace('/','_')

'pablocosta_bertabaporu-base-uncased'

In [8]:
for exp_name, config in dict_experiments.items():
    
    print(f"""####################################  
# Running {exp_name}
#####################################""")
    
    
    # get configs of experiments
    text_col = config['text_col']
    file_format = config['file_format']
    n_comments = config['n_comments']
    file_format = config['file_format']
    
    
    list_responses = []
    for target in target_list:
        
        print(target)
        
        # read data
        train = pd.read_csv(
            file_format.format(target = target, split = "train"), 
            sep = ';', 
            encoding='utf-8-sig'
            )

        train[f'comments_and_scores_{text_col}'] = train[f'comments_and_scores_{text_col}'].progress_apply(lambda x: literal_eval(x))

        train[text_col] = train[f'comments_and_scores_{text_col}'].progress_apply(
            lambda x: " # ".join([comment for score, comment in x[-n_comments:]])
            ) 
        train.Polarity = train.Polarity.map({
            "against": False,
            "for": True
        })
        test = pd.read_csv(
            file_format.format(target = target, split = "test"), 
            sep = ';', 
            encoding='utf-8-sig'
            )

        test[f'comments_and_scores_{text_col}'] = test[f'comments_and_scores_{text_col}'].progress_apply(lambda x: literal_eval(x))

        test[text_col] = test[f'comments_and_scores_{text_col}'].progress_apply(
            lambda x: " # ".join([comment for score, comment in x[-n_comments:]])
            ) 

        test.Polarity = test.Polarity.map({
            "against": False,
            "for": True
        })
        
        
        X_train = train[text_col].tolist()
        X_test = test[text_col].tolist()

        y_train = train["Polarity"].tolist()
        y_test = test["Polarity"].tolist()        
        
        torch.cuda.empty_cache()
        
        model = BertClassifierWithPooling(**MODEL_PARAMS, device="cuda:0")
        
        model.fit(X_train, y_train, epochs=epochs)
        
        y_pred = model.predict_classes(X_test)
        
        del model
        
        df_responses = pd.DataFrame({
            "y_test": y_test,
            "y_pred": y_pred
        })
        
        df_responses['target'] =target
        df_responses['exp_name'] = exp_name
        df_responses['n_comments'] = n_comments
        df_responses['text_col'] = text_col
        
        print(classification_report(y_test, y_pred))
        df_responses.to_csv(f'{reports_path}test_results/belt_{exp_name}_{bert_model_name.replace('/','_')}_test_results_part_{target}.csv')  
        
        list_responses.append(df_responses)
        
df_responses_final = pd.concat(list_responses)
df_responses_final.to_csv(f'{reports_path}test_results/belt_{exp_name}_{bert_model_name.replace('/','_')}_test_results.csv')


####################################  
# Running filtered_Texts5
#####################################
ig


100%|██████████| 599/599 [00:00<00:00, 352319.18it/s]


              precision    recall  f1-score   support

       False       0.66      0.61      0.63       339
        True       0.54      0.60      0.57       260

    accuracy                           0.60       599
   macro avg       0.60      0.60      0.60       599
weighted avg       0.61      0.60      0.61       599

bo


100%|██████████| 188/188 [00:00<00:00, 260584.65it/s]
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

              precision    recall  f1-score   support

       False       0.86      1.00      0.93       162
        True       0.00      0.00      0.00        26

    accuracy                           0.86       188
   macro avg       0.43      0.50      0.46       188
weighted avg       0.74      0.86      0.80       188

cl


...

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       354
        True       0.54      1.00      0.70       420

    accuracy                           0.54       774
   macro avg       0.27      0.50      0.35       774
weighted avg       0.29      0.54      0.38       774

gl


100%|██████████| 411/411 [00:00<00:00, 228325.69it/s]
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisi

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       167
        True       0.59      1.00      0.75       244

    accuracy                           0.59       411
   macro avg       0.30      0.50      0.37       411
weighted avg       0.35      0.59      0.44       411

lu


100%|██████████| 272/272 [00:00<00:00, 208947.01it/s]


              precision    recall  f1-score   support

       False       0.74      0.89      0.81       143
        True       0.84      0.65      0.73       129

    accuracy                           0.78       272
   macro avg       0.79      0.77      0.77       272
weighted avg       0.79      0.78      0.77       272



In [11]:
df_responses_final['hit'] = df_responses_final.apply(lambda x: 1 if x.y_test == x.y_pred else 0,axis = 1)
df_responses_final

,y_test,y_pred,target,exp_name,n_comments,text_col,hit
0,False,True,ig,filteredTimeline15,15,Timeline,0
1,False,False,ig,filteredTimeline15,15,Timeline,1
2,False,True,ig,filteredTimeline15,15,Timeline,0
3,True,True,ig,filteredTimeline15,15,Timeline,1
4,True,True,ig,filteredTimeline15,15,Timeline,1
...,...,...,...,...,...,...,...
267,True,True,lu,filteredTimeline15,15,Timeline,1
268,False,False,lu,filteredTimeline15,15,Timeline,1
269,True,False,lu,filteredTimeline15,15,Timeline,0
270,True,True,lu,filteredTimeline15,15,Timeline,1


In [13]:
df_responses_final

,y_test,y_pred,target,exp_name,n_comments,text_col,hit
0,False,True,ig,filteredTimeline15,15,Timeline,0
1,False,False,ig,filteredTimeline15,15,Timeline,1
2,False,True,ig,filteredTimeline15,15,Timeline,0
3,True,True,ig,filteredTimeline15,15,Timeline,1
4,True,True,ig,filteredTimeline15,15,Timeline,1
...,...,...,...,...,...,...,...
267,True,True,lu,filteredTimeline15,15,Timeline,1
268,False,False,lu,filteredTimeline15,15,Timeline,1
269,True,False,lu,filteredTimeline15,15,Timeline,0
270,True,True,lu,filteredTimeline15,15,Timeline,1


In [12]:
df_responses_final.groupby('n_comments').mean('hit')

,y_test,y_pred,hit
n_comments,,,
15,0.484031,0.669624,0.675302
